## 1. Import necessary libraries for loading our data


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

## 2. Define and intialize the neural network


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # Second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(32, 64 , 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        # First fully connected layer
        self.fc1 = nn.Linear(9216, 128)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(128, 10)

In [4]:

my_nn = Net()
print(my_nn)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


## 3. Specify how data will pass through your model


In [41]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # Second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(32, 64 , 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        # First fully connected layer
        self.fc1 = nn.Linear(9216, 128)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(128, 10)


    # x represents our data
    def forward(self, x):
        # Pass data through conv1
        x = self.conv1(x)
        print("size after conv1 : {}".format(x.size()))
        # Use the rectified-linear activation function over x
        x = F.relu(x)

        x = self.conv2(x)
        print("size after conv2 : {}".format(x.size()))
        x = F.relu(x)

        # Run max pooling over x
        x = F.max_pool2d(x, 2)
        # Pass data through dropout1
        x = self.dropout1(x)
        print("size after maxpool : {}".format(x.size()))
        # Flatten x with start_dim=1
        x = torch.flatten(x, 1)
        print("size after flatten : {}".format(x.size()))
        # Pass data through fc1
        x = self.fc1(x)
        print("size after fc1 : {}".format(x.size()))
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        print("size after fc2 : {}".format(x.size()))

        # Apply softmax to x
        output = F.log_softmax(x, dim=1)
        return output

In [42]:
# Equates to one random 28x28 image
random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(random_data)
print (result)

size after conv1 : torch.Size([1, 32, 26, 26])
size after conv2 : torch.Size([1, 64, 24, 24])
size after maxpool : torch.Size([1, 64, 12, 12])
size after flatten : torch.Size([1, 9216])
size after fc1 : torch.Size([1, 128])
size after fc2 : torch.Size([1, 10])
tensor([[-2.5122, -2.4271, -2.1249, -2.3011, -2.2669, -2.2030, -2.1566, -2.3981,
         -2.3268, -2.3783]], grad_fn=<LogSoftmaxBackward>)


In [43]:
summary(my_nn,(1, 28, 28))

size after conv1 : torch.Size([2, 32, 26, 26])
size after conv2 : torch.Size([2, 64, 24, 24])
size after maxpool : torch.Size([2, 64, 12, 12])
size after flatten : torch.Size([2, 9216])
size after fc1 : torch.Size([2, 128])
size after fc2 : torch.Size([2, 10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
         Dropout2d-3           [-1, 64, 12, 12]               0
            Linear-4                  [-1, 128]       1,179,776
         Dropout2d-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.52
Params size (MB): 4.58
Es